In [1]:
from mtcnn import MTCNN
from cv2 import imread
from PIL import Image
import numpy as np
import os
from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
import cv2

warnings.filterwarnings("ignore")


def extract_face(
    filename=None, image_pixels=None, required_size=(160, 160), use_haarcascades=False
):
    if filename is not None:
        image = imread(filename)
        image = Image.fromarray(image)
        pixels = np.asarray(image)
    elif image_pixels is not None:
        pixels = image_pixels

    if use_haarcascades:
        gray = cv2.cvtColor(pixels, cv2.COLOR_RGB2GRAY)
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + r"K:/FaceRecognization/haarcascade_frontalface_default.xml"
        )
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        if len(faces) == 0:
            return None, None

        x, y, w, h = faces[0]
        face = pixels[y : y + h, x : x + w]
        box_dimensions = (x, y, w, h)

    else:
        detector = MTCNN()
        results = detector.detect_faces(pixels)
        if not results:
            return None, None

        x1, y1, width, height = results[0]["box"]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        box_dimensions = (x1, y1, width, height)

    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array, box_dimensions


def load_faces(directory, use_haarcascades=False):
    faces = []
    labels = []
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        face, _ = extract_face(filename=path, use_haarcascades=use_haarcascades)
        if face is not None:
            faces.append(face)
            labels.append(
                directory.split("/")[-1]
            )  # Assuming directory structure is like '.../data/class_name/'
    return faces, labels


def load_dataset(directory, use_haarcascades=False):
    X, y = [], []
    for subdir in os.listdir(directory):
        path = os.path.join(directory, subdir)
        if not os.path.isdir(path):
            continue
        faces, labels = load_faces(path, use_haarcascades)
        print(f"Loaded {len(faces)} examples for class: {subdir}")
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)


# Load train dataset
X_train, y_train = load_dataset(
    r"M:/14-celebrity-faces-dataset/data/train", use_haarcascades=True
)
print(f"Train Dataset Shape: {X_train.shape}, Labels Shape: {y_train.shape}")

# Load validation dataset
X_val, y_val = load_dataset(
    r"M:/14-celebrity-faces-dataset/data/val", use_haarcascades=True
)
print(f"Validation Dataset Shape: {X_val.shape}, Labels Shape: {y_val.shape}")

# Load the FaceNet model with pre-trained weights
model = FaceNet()

# Convert each face in the training set to an embedding
train_embeddings = []
for face_pixels in X_train:
    embedding = model.model.predict(np.expand_dims(face_pixels / 255.0, axis=0))[0]
    train_embeddings.append(embedding)
train_embeddings = np.asarray(train_embeddings)
print(f"Train Embeddings Shape: {train_embeddings.shape}")

# Convert each face in the validation set to an embedding
val_embeddings = []
for face_pixels in X_val:
    embedding = model.model.predict(np.expand_dims(face_pixels / 255.0, axis=0))[0]
    val_embeddings.append(embedding)
val_embeddings = np.asarray(val_embeddings)
print(f"Validation Embeddings Shape: {val_embeddings.shape}")

# Label encoding for SVM
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

# Train SVM classifier
svm_classifier = SVC(kernel="linear", probability=True)
svm_classifier.fit(train_embeddings, y_train_encoded)

# Predict on validation set
y_val_pred = svm_classifier.predict(val_embeddings)
accuracy = accuracy_score(y_val_encoded, y_val_pred)
print(f"Validation Accuracy: {accuracy}")

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


In [2]:
import random

street_prefixes = [
    "1st",
    "2nd",
    "3rd",
    "4th",
    "5th",
    "6th",
    "7th",
    "8th",
    "9th",
    "10th",
    "Main",
    "Elm",
    "Oak",
    "Pine",
    "Maple",
    "Hill",
    "Lake",
    "River",
    "Park",
]
street_suffixes = ["Street", "Avenue", "Road", "Lane", "Drive", "Place", "Court", "Way"]


def generate_address():
    house_number = str(random.randint(100, 99999))
    street_prefix = random.choice(street_prefixes)
    street_suffix = random.choice(street_suffixes)
    cross_street = (
        str(random.randint(1, 20)) + random.choice(["st", "nd", "rd", "th"]) + " Cross"
    )

    # Determine if there's a comma and cross street
    if random.random() > 0.3:  # 70% chance of having no comma/cross street
        return f"{house_number}, {street_prefix} {street_suffix}"
    else:
        return f"{house_number}, {street_prefix} {street_suffix}, {cross_street}"


for _ in range(50):  # Generate 5 addresses
    print(generate_address())

96164, Maple Place, 10nd Cross
31327, Main Place
57615, Park Avenue
78616, Hill Way
45896, 4th Drive
8263, 7th Place
43067, Oak Road
15003, Oak Way
32567, River Avenue
23825, 4th Way
5381, Oak Avenue, 14nd Cross
6522, 9th Place
64614, Elm Road
26575, 3rd Street
35433, 1st Avenue
3815, 7th Lane, 7rd Cross
28877, 7th Drive
97104, 2nd Place
49315, Elm Place, 11st Cross
94309, 7th Lane
58064, 6th Drive
42141, Elm Drive
86802, 9th Way
86321, 6th Avenue
83946, Maple Court
93713, Lake Road
90575, 9th Street
87781, Main Drive
4877, Hill Road
30295, 8th Way, 13st Cross
16489, Main Place, 8rd Cross
112, 10th Avenue
28607, Lake Court
23575, Maple Way, 15rd Cross
86397, Maple Road
56683, 1st Lane
56323, 4th Street, 2rd Cross
15986, 7th Road, 1rd Cross
69558, Lake Drive
76142, 4th Place, 1nd Cross
93629, Maple Road
36298, 6th Avenue, 1st Cross
57968, 6th Court, 3th Cross
89155, 4th Street
14361, Park Place, 2nd Cross
19064, River Road, 20nd Cross
99533, River Street
66792, Lake Way, 5nd Cross
21712